In [6]:
import cv2
import numpy as np
import math
import tkinter as tk
from tkinter import filedialog, messagebox


def embed_message(image_path, message, output_path):
    # Görüntüyü oku ve gri tonlamaya çevir
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Görüntü okunamadı.")
    rows, cols = image.shape
    n_blocks = rows // 8 * cols // 8

    # Mesajı bit dizisine çevir
    message += '###'  # Mesaj sonu işareti
    message_bits = ''.join(f'{ord(c):08b}' for c in message)
    if len(message_bits) > n_blocks:
        raise ValueError("Mesaj çok uzun, görüntüye sığmıyor.")

    # Bloklara ayır ve DCT uygula
    stego_image = np.copy(image)
    bit_idx = 0
    for i in range(0, rows, 8):
        for j in range(0, cols, 8):
            if bit_idx >= len(message_bits):
                break
            block = stego_image[i:i+8, j:j+8]
            dct_block = cv2.dct(np.float32(block))
            # (1,1) katsayısının LSB'sini mesaj bitiyle değiştir
            coeff = dct_block[1,1]
            coeff = int(coeff)
            coeff = (coeff & ~1) | int(message_bits[bit_idx])
            dct_block[1,1] = coeff
            idct_block = cv2.idct(dct_block)
            stego_image[i:i+8, j:j+8] = np.uint8(idct_block)
            bit_idx += 1
        if bit_idx >= len(message_bits):
            break

    # Yeni görüntüyü kaydet
    cv2.imwrite(output_path, stego_image)

def extract_message(stego_path):
    # Görüntüyü oku ve gri tonlamaya çevir
    image = cv2.imread(stego_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Görüntü okunamadı.")
    rows, cols = image.shape

    # Bloklardan mesajı çıkar
    bits = ''
    for i in range(0, rows, 8):
        for j in range(0, cols, 8):
            block = image[i:i+8, j:j+8]
            dct_block = cv2.dct(np.float32(block))
            coeff = int(dct_block[1,1])
            bits += str(coeff & 1)
            if len(bits) % 8 == 0:
                char = chr(int(bits[-8:], 2))
                if bits.endswith('001000110010001100100011'):  # '###' bit dizisi
                    message = ''.join(chr(int(bits[i:i+8], 2)) for i in range(0, len(bits)-24, 8))
                    return message
    return ''


In [7]:
def select_image():
    path = filedialog.askopenfilename(filetypes=[("JPEG files", "*.jpg;*.jpeg")])
    entry_image.delete(0, tk.END)
    entry_image.insert(0, path)

def save_image():
    path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg;*.jpeg")])
    entry_output.delete(0, tk.END)
    entry_output.insert(0, path)

def hide_message():
    image_path = entry_image.get()
    output_path = entry_output.get()
    message = entry_message.get()
    if len(message) > 160:
        messagebox.showerror("Hata", "Mesaj 160 karakterden uzun olamaz.")
        return
    try:
        embed_message(image_path, message, output_path)
        messagebox.showinfo("Başarılı", "Mesaj başarıyla gizlendi.")
    except Exception as e:
        messagebox.showerror("Hata", str(e))

def reveal_message():
    image_path = entry_image.get()
    try:
        message = extract_message(image_path)
        messagebox.showinfo("Gizli Mesaj", message)
    except Exception as e:
        messagebox.showerror("Hata", str(e))


In [8]:
root = tk.Tk()
root.title("JPEG DCT Steganografi")

tk.Label(root, text="Giriş Görüntüsü:").grid(row=0, column=0, sticky="e")
entry_image = tk.Entry(root, width=50)
entry_image.grid(row=0, column=1)
tk.Button(root, text="Seç", command=select_image).grid(row=0, column=2)

tk.Label(root, text="Çıkış Görüntüsü:").grid(row=1, column=0, sticky="e")
entry_output = tk.Entry(root, width=50)
entry_output.grid(row=1, column=1)
tk.Button(root, text="Kaydet", command=save_image).grid(row=1, column=2)

tk.Label(root, text="Mesaj:").grid(row=2, column=0, sticky="e")
entry_message = tk.Entry(root, width=50)
entry_message.grid(row=2, column=1, columnspan=2)

tk.Button(root, text="Mesajı Gizle", command=hide_message).grid(row=3, column=1)
tk.Button(root, text="Mesajı Göster", command=reveal_message).grid(row=3, column=2)

root.mainloop()